In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2
import random
import time


from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.utils import to_categorical
from keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.regularizers import l2
from keras import backend as K
#from keras.engine import Layer, InputSpec

from sklearn.utils import shuffle

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#/kaggle/input/indian-food-images-dataset/List of Indian Foods.txt
#/kaggle/input/indian-food-images-dataset/Indian Food Images/Indian Food Images/mysore_pak/
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
 #       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_dir = "D:/Github/Datasets/Indian Food Images/Indian Food Images"

In [3]:
food_names = os.listdir(train_dir)
food_names

['adhirasam',
 'aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'anarsa',
 'ariselu',
 'bandar_laddu',
 'basundi',
 'bhatura',
 'bhindi_masala',
 'biryani',
 'boondi',
 'butter_chicken',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chhena_kheeri',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chikki',
 'daal_baati_churma',
 'daal_puri',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'doodhpak',
 'double_ka_meetha',
 'dum_aloo',
 'gajar_ka_halwa',
 'gavvalu',
 'ghevar',
 'gulab_jamun',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kajjikaya',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'kofta',
 'kuzhi_paniyaram',
 'lassi',
 'ledikeni',
 'litti_chokha',
 'lyangcha',
 'maach_jhol',
 'makki_di_roti_sarson_da_saag',
 'malapua',
 'misi_roti',
 'misti_doi',
 'modak',
 'mysore_pak',
 'naan',
 'navrattan_korma',
 'palak_paneer',
 'paneer_butter_masala',
 'phirni',
 'pithe',
 'poha',
 'poornalu

In [4]:
np.array(food_names)

array(['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi',
       'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu',
       'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani',
       'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham',
       'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala',
       'chicken_tikka', 'chicken_tikka_masala', 'chikki',
       'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka',
       'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo',
       'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti',
       'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya',
       'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta',
       'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha',
       'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua',
       'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan',
       'navrattan_korma', 'palak_paneer', 'paneer_butter_masala

In [5]:
print(len(np.array(food_names)))

80


In [6]:
len(food_names)

80

All the 80 classes will be used to train the siamese model.

Images 1-40 in each class will be used to train the model.

Images 41-50 will be used to validated the model.

In [7]:
IMG_Shape = (200,200,3)
train_ids = np.arange(0,35)
val_ids = np.arange(35,45)
test_ids = np.arange(45,50)

In [8]:
train_ids

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34])

In [9]:
def loadimgs(names,n = 0,dataset="train"):
    '''
    path => Path of train directory or test directory
    '''
    X_train=[]
    X_val = []
    y_train = []
    y_val =[]
    X_test = []
    y_test =[]
    
    img_shape = (200,200)
    cat_dict = {}
    dish_dict = {}
    curr_y = n
    path = 'D:/Github/Datasets/Indian Food Images/Indian Food Images'
    
    category_images_tr=[]
    category_images_v=[]
    category_images_te=[] 
        
    clsctr = 0
    for dish in names:
        dish_dict[dish] = clsctr
        dish_path = os.path.join(path,dish) +'/'
        
        #category_images_t=[]
        #category_images_v=[]
                  
        #count = 0 
        #print(len(os.listdir(dish_path)))
        for count, filename in enumerate(os.listdir(dish_path)):
            #print("Class :{}, count: {}".format(clsctr,count)) 
            if count < 35:
                #print("In train condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image #cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                y_train.append(clsctr)
                try:
                    category_images_tr.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_train:", final_image)
    
                
            elif count >= 35 and count < 45:
                #print("In val condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image#cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                y_val.append(clsctr)
                try:
                    category_images_v.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_test:", final_image)
             
            
            else :
                #print("In test condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image#cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                y_test.append(clsctr)
                try:
                    category_images_te.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_test:", final_image)
             
            curr_y += 1    
             
             #count = count + 1
            
        #X_train.append(category_images_t)
        #X_val.append(category_images_v)
        clsctr += 1
    
    #y_train = np.vstack(y_train)
    #y_val = np.vstack(y_val)
    #X_train = np.stack(X_train)
    #X_val = np.stack(X_val)
    X_train = category_images_tr
    X_val = category_images_v
    X_test = category_images_te
    
    #print(y_train)
    
    return X_train,y_train, X_val,y_val, X_test,y_test, dish_dict

In [10]:
X_train, y_train,X_val,y_val,X_test,y_test, dish_dict = loadimgs(food_names)

In [11]:
len(X_train), len(X_val), len(y_train), len(y_val), len(X_test), len(y_test)

(2800, 800, 2800, 800, 400, 400)

In [12]:
def create_pairs(images, labels):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    random.seed(2024)
    pairImages = []
    pairLabels = []
   
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    numClasses = len(np.unique(y_val))
    classes=np.unique(y_val)
    idx = [np.where(y_val == classes[i]) for i in range(0, numClasses)]
    
    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current iteration
        currentImage = images[idxA]
        label = labels[idxA]
        
        # randomly pick an image that belongs to the *same* class
        # label
        #posId = random.choice(list(np.where(labels == label)))
        posIdx =random.choice([index for index, element in enumerate(labels) if element == label])
        posImage = images[posIdx]
        
        # prepare a positive pair and update the images and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])
        
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        #negId = random.choice(list(np.where(labels != label)))         
        negIdx =random.choice([index for index, element in enumerate(labels) if element != label])
        negImage = images[negIdx]
        
        # prepare a negative pair of images and update our lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
   
    return (np.array(pairImages), np.array(pairLabels))

In [13]:
(pairTrain, labelTrain) = create_pairs(X_train, y_train)
(pairval, labelval) = create_pairs(X_val, y_val)
(pairTest, labelTest) = create_pairs(X_test, y_test)

In [14]:
def tf_siamese_nn(shape, embedding=64, fineTune=False):
    inputs = tf.keras.layers.Input(shape)
    preprocess_fn = preprocess_input
    base_model = tf.keras.applications.vgg19.VGG19(input_shape=shape, include_top=False,                                               weights='imagenet')
    
    if fineTune==False:
        base_model.trainable=False
    else:
        base_model.trainable = True
        # Fine-tune from this layer onwards
        fine_tune_at = len(base_model.layers)-int(len(base_model.layers)*.10)
# Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
          layer.trainable =  False
    x=base_model(inputs)
    x=tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs=tf.keras.layers.Dense(embedding)(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

model1=tf_siamese_nn(IMG_Shape, 64, True)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 6, 6, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
Total params: 20,057,216
Trainable params: 2,392,640
Non-trainable params: 17,664,576
_________________________________________________________________


In [15]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [16]:
img1 = tf.keras.layers.Input(shape=IMG_Shape)
img2 =  tf.keras.layers.Input( shape=IMG_Shape)
featureExtractor = tf_siamese_nn(IMG_Shape)
featsA = featureExtractor(img1)
featsB = featureExtractor(img2)

In [17]:
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

In [18]:
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
model = tf.keras.Model(inputs=[img1, img2], outputs=outputs)

In [19]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:], validation_data=([pairval[:, 0], pairval[:, 1]], labelval[:]), batch_size=1, epochs=20)

Epoch 1/20
5600/5600 [==============================] - 292s 51ms/step - loss: 3.1819 - accuracy: 0.5870 - val_loss: 0.6066 - val_accuracy: 0.6775
Epoch 2/20
5600/5600 [==============================] - 289s 52ms/step - loss: 0.5854 - accuracy: 0.6971 - val_loss: 0.5717 - val_accuracy: 0.6875
Epoch 3/20
5600/5600 [==============================] - 290s 52ms/step - loss: 0.5492 - accuracy: 0.7316 - val_loss: 0.5647 - val_accuracy: 0.6975
Epoch 4/20
5600/5600 [==============================] - 290s 52ms/step - loss: 0.5084 - accuracy: 0.7579 - val_loss: 0.5692 - val_accuracy: 0.6931
Epoch 5/20
5600/5600 [==============================] - 289s 52ms/step - loss: 0.4749 - accuracy: 0.7845 - val_loss: 0.6253 - val_accuracy: 0.6687
Epoch 6/20
5600/5600 [==============================] - 290s 52ms/step - loss: 0.4466 - accuracy: 0.7989 - val_loss: 0.5905 - val_accuracy: 0.6938
Epoch 7/20
5600/5600 [==============================] - 290s 52ms/step - loss: 0.4183 - accuracy: 0.8195 - val_loss: 0

In [21]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "iris_classifier_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)

INFO:tensorflow:Assets written to: ram://e580f7d9-6d51-40d6-9cc2-b42b22413ed0/assets


INFO:tensorflow:Assets written to: ram://e580f7d9-6d51-40d6-9cc2-b42b22413ed0/assets


In [22]:
preds = model.predict( [pairTest[:,0],pairTest[:,1]])

25/25 [==============================] - 25s 521ms/step


In [23]:
preds_ = [int(i > .5) for i in preds]

In [24]:
labelTest = labelTest.flatten()

In [25]:
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

In [26]:
compute_accuracy(labelTest,preds_)

0.74

In [28]:
def loadimgs_(names):
    '''
    path => Path of train directory or test directory
    '''
    images = []
    labels = [] 
    
    img_shape = (200,200)
    dish_dict = {}
    
    path = 'D:/Github/Datasets/Indian Food Images/Indian Food Images'
    
    category_images = []
        
    clsctr = 0
    for dish in names:
        dish_dict[dish] = clsctr
        dish_path = os.path.join(path,dish) +'/'
        
        for count, filename in enumerate(os.listdir(dish_path)):
            #print("Class :{}, count: {}".format(clsctr,count)) 
            if count == 1:
                #print("In train condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image #cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                labels.append(clsctr)
                try:
                    category_images.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_train:", final_image)
            else:
                pass

        clsctr += 1
        
    images = category_images
    
    return images, labels, dish_dict

In [29]:
images_, labels_ , dish_dict_ = loadimgs_(food_names) 

In [30]:
len(images_)

80

In [31]:
def create_pairs_for_pred(image_path, images_):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    random.seed(2024)
    pairImages = []
    pairLabels = []
   
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (200,200))
                
    # loop over all images
    for idxA in range(len(images_)):
    
        currentImage = images_[idxA]

        pairImages.append([currentImage, resized_image]) 
   
    return (np.array(pairImages))

In [32]:
os.listdir('D:/Github/Datasets/Indian Food Images/Indian Food Images/rasgulla/')

['014ce94539.jpg',
 '03c77ada4d.jpg',
 '063a16b50b.jpg',
 '08cca5782a.jpg',
 '11a818e3f6.jpg',
 '12dace47fc.jpg',
 '15e2514789.jpg',
 '15f52d7785.jpg',
 '1b02eccfbe.jpg',
 '1dbc60a00c.jpg',
 '1e3f3359d5.jpg',
 '2ce50a96d1.jpg',
 '2fff4054b6.jpg',
 '3a1266de50.jpg',
 '3a4924c5f0.jpg',
 '3ac546bb60.jpg',
 '3b11e111b7.jpg',
 '3c22b84dd9.jpg',
 '3d24434b7a.jpg',
 '3d4802b49c.jpg',
 '3e474c7fa0.jpg',
 '3fa1c081db.jpg',
 '4efcadced4.jpg',
 '55aa0e6d96.jpg',
 '55f603e47f.jpg',
 '5a200c3472.jpg',
 '5cd440e52d.jpg',
 '5f675d88e0.jpg',
 '67a0b5d362.jpg',
 '69dd42b748.jpg',
 '6aa91b4eea.jpg',
 '6ac863dbb4.jpg',
 '6d71ed5332.jpg',
 '6d92c18226.jpg',
 '6e370baada.jpg',
 '6f58b256d5.jpg',
 '70b595e1ac.jpg',
 '72a87c23a2.jpg',
 '79bc9bded9.jpg',
 '7c6367c1e6.jpg',
 '81be6155a9.jpg',
 '82bf2b6314.jpg',
 '84b6fcd406.jpg',
 '86f1c46bd8.jpg',
 '87d7c05399.jpg',
 '8ea9cbf7ee.jpg',
 '9a26b97d4b.jpg',
 '9c0002e783.jpg',
 '9d60bf1eb5.jpg',
 '9edcc97fc8.jpg']

In [33]:
image_path ='D:/Github/Datasets/Indian Food Images/Indian Food Images/rasgulla/55aa0e6d96.jpg'
pairPred = create_pairs_for_pred(image_path, images_)

In [34]:
image_prediction = model.predict( [pairPred[:,0],pairPred[:,1]])

3/3 [==============================] - 9s 4s/step


In [35]:
print("The image is of {}".format(list(dish_dict.keys())[list(dish_dict.values()).index(np.argmax(image_prediction))])) 

The image is of rasgulla


In [39]:
def predict(image_path):
    pairPred = create_pairs_for_pred(image_path, images_)
    image_prediction = model.predict( [pairPred[:,0],pairPred[:,1]])
    return "The image is of {}".format(list(dish_dict.keys())[list(dish_dict.values()).index(np.argmax(image_prediction))])

predict("D:/Github/Datasets/Indian Food Images/Indian Food Images/butter_chicken/2ab5e4e96a.jpg")

3/3 [==============================] - 1s 537ms/step


'The image is of chicken_tikka_masala'